## Тема “Создание признакового пространства” 

### Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'. 

In [1]:
import pandas as pd
import numpy as np

import re

import string
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import spacy
import nltk
from nltk.tokenize import word_tokenize

from gensim.models import Word2Vec

import warnings 
warnings.filterwarnings("ignore")
import os

#### Задание 1. 

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000) 

С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор? 

Повторим шаги из заданий 1 и 2, используя библиотеку nltk. 

In [2]:
processed_df = pd.read_pickle('processed_df.pkl')
processed_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


In [3]:
%%time
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")
processed_df['NER'] = processed_df['clean_tweet'].apply(lambda x:  nlp(x))

ner = processed_df['NER'].tolist()

NER = []
for doc in ner:
    for ent in doc.ents:
        NER.append((ent.text, ent.label_))

df_ner = pd.DataFrame(NER, columns=['word', 'ner'])
df_ner.head()

Wall time: 4min 30s


,word,ner
0,wheelchair vans,ORG
1,pdx,ORG
2,bihday,PERSON
3,tomorrow,DATE
4,the next school year,DATE


#### ТОП-20 популярных NER

In [4]:
top_20_ner = df_ner.ner.value_counts().head(20)
top_20_ner

DATE           11507
PERSON          8040
ORG             5575
GPE             4570
TIME            2021
NORP            1448
CARDINAL        1098
ORDINAL          647
FAC              303
LOC              216
EVENT            151
PRODUCT           63
LANGUAGE          43
QUANTITY          34
WORK_OF_ART       27
LAW               21
MONEY              8
PERCENT            3
Name: ner, dtype: int64

#### Топ-20 персон

In [5]:
top_20_pers = df_ner.loc[df_ner['ner'] == 'PERSON']

top_20_pers = top_20_pers.word.value_counts().head(20)
top_20_pers

hea                                                        116
hu                                                          83
blur sun                                                    54
bihday                                                      46
feminismiscancer feminismisterrorism feminismmuktbharat     40
christina grimmie                                           38
hillary                                                     36
sikh temple                                                 28
tgif ff                                                     26
don                                                         24
clinton                                                     21
detoxdiet altwaystoheal                                     21
ripchristina                                                20
jo cox                                                      20
carl paladino                                               19
regrann                                                

#### Тот-20 организаций

In [6]:
top_20_org = df_ner.loc[df_ner['ner'] == 'ORG']

top_20_org = top_20_org.word.value_counts().head(20)
top_20_org

bong bing           107
app                  84
gop                  77
islam                58
house                46
social analytics     40
nba                  39
usa                  34
amazon               32
sta                  31
sma                  30
cnn                  28
euro                 26
fed                  25
ios                  23
eu                   22
fbi                  21
sun                  20
bogota colombia      20
congress             19
Name: word, dtype: int64

#### Задание 2. 

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру. 

С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор? 

In [7]:
%%time
exclude = set(string.punctuation)
stop_words = set(get_stop_words("en"))
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = re.sub('@[\w]*', ' ', txt)
    txt = re.sub(r'[^\w\s]',' ', txt)
    txt = re.sub(r'[0-9]+', ' ', txt)
    txt = re.sub('\n', ' ', txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = [word for word in txt.split() if word not in exclude if word not in stop_words]
    return " ".join(txt)


processed_df['tagged'] = processed_df['tweet'].apply(preprocess_text).apply(
    lambda x: nltk.pos_tag(nltk.word_tokenize(x)) 
)

processed_df['tagged'].head()

Wall time: 44.8 s


0    [(father, RBR), (dysfunctional, JJ), (selfish,...
1    [(thanks, NNS), (lyft, VBP), (credit, NN), (ca...
2                        [(bihday, NN), (majesty, NN)]
3    [(model, NN), (love, NN), (u, JJ), (take, VB),...
4    [(factsguide, JJ), (society, NN), (now, RB), (...
Name: tagged, dtype: object

In [8]:
corpus = []
for text in processed_df['tagged'].tolist():
    corpus.extend(text)

In [9]:
len(corpus)

442180

In [10]:
ner = {
    (' '.join(c[0] for c in chunk), chunk.label()) 
    for chunk in nltk.ne_chunk(corpus) 
    if hasattr(chunk, 'label')
}

In [11]:
ner

{('adriÃ', 'ORGANIZATION'),
 ('aldilÃ', 'ORGANIZATION'),
 ('anaÃ', 'ORGANIZATION'),
 ('anchequestoÃ', 'ORGANIZATION'),
 ('andrÃ', 'ORGANIZATION'),
 ('areÂ', 'ORGANIZATION'),
 ('asÃ', 'ORGANIZATION'),
 ('atÂ', 'ORGANIZATION'),
 ('babalargÃ¼nÃ¼', 'ORGANIZATION'),
 ('befÄ', 'ORGANIZATION'),
 ('bellÃ', 'ORGANIZATION'),
 ('beyoncÃ', 'ORGANIZATION'),
 ('beyoÄ', 'ORGANIZATION'),
 ('bonnejournÃ', 'ORGANIZATION'),
 ('braggingÂ', 'ORGANIZATION'),
 ('brÃ¼ssel', 'ORGANIZATION'),
 ('cafÃ', 'ORGANIZATION'),
 ('canÂ', 'ORGANIZATION'),
 ('castaÃ', 'ORGANIZATION'),
 ('catedraldesaldezipaquirÃ', 'ORGANIZATION'),
 ('chloÃ', 'ORGANIZATION'),
 ('chÃ', 'ORGANIZATION'),
 ('citlaltÃ', 'ORGANIZATION'),
 ('comounniÃ', 'ORGANIZATION'),
 ('coolÂ', 'ORGANIZATION'),
 ('costeÃ', 'ORGANIZATION'),
 ('crÃªpes', 'ORGANIZATION'),
 ('cumpleaÃ', 'ORGANIZATION'),
 ('deberÃ de', 'ORGANIZATION'),
 ('depresiÃ³n', 'ORGANIZATION'),
 ('diplomalÄ', 'ORGANIZATION'),
 ('diversiÃ³n', 'ORGANIZATION'),
 ('divulgaÃ Ã', 'ORGANIZATION'),


#### Задание 3. 

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER. 

 